---

### **Import SK packages**

In [ ]:
#r "nuget:Microsoft.SemanticKernel, 0.16.230615.1-preview"
//#r "nuget:Microsoft.SemanticKernel, 0.17.230718.1-preview"



---

### **Load secrets into local variables**

In [ ]:
#!import ../Settings/AppConfig.cs

//#!import /Settings/AppConfig.cs

AppConfig.aoaiSettings apiConfig = AppConfig.GetSecrets();

// Get AOAI values
string mydeployDavinci = apiConfig.deployDavinci;
string mydeployEmbed = apiConfig.deployEmbed; 
string myendpoint = apiConfig.endpoint;
string myAPIKey = apiConfig.APIKey;
string myQdrantEndpoint = apiConfig.QdrantEndpoint;

//NOTE: If you run the below writeline command, be sure to clear the output before pushing to repo!!
//Console.WriteLine(mydeployDavinci + " & " + mydeployEmbed + " & " + myQdrantEndpoint + " & " + myendpoint + " & " + myAPIKey);

---
### **Build the Semantic Kernel client**

In [ ]:
using Microsoft.SemanticKernel;
//using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
//using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

kernel.Config.AddAzureTextCompletionService(mydeployDavinci, myendpoint, myAPIKey);

---
### **Load Skills and setup test input**

In [ ]:
using System.Text.Json;
using System.Text.Json.Serialization;

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

// The default input variable 
var input = @"
Customer 1: The power button on my phone is broken. The warranty is still valid.
Customer 2: My display stopped working.
Customer 3: The customer service rep didn't answer my email.
Customer 4: Every time I call customer support I get no answer.
Customer 5: The display screen cracked and it's still under warranty.
Customer 6: My power button fell off the phone. That's ridiculous.
Customer 7: I'm so frustrated with this company.
Customer 8: When I use the power button too much, it stops working.
";

Console.WriteLine(input);

---
### **Example using Stanford University's "Design Thinking"**
#### Empathize, Define, Ideate, Prototype, Test.
<img src="designthinking.png" width="50%" height="50%">

#### Let's run the "Empathize" step to understand our customers' painpoints.



In [ ]:
var empathyResult = await kernel.RunAsync(input, skillDT["Empathize"]);

Console.WriteLine(empathyResult);

---
#### Turn the empathy insights into problem definitions with the "Define" step

In [ ]:
// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

var defineResult = await kernel.RunAsync(empathyResult.ToString(), skillDT["Define"]);

Console.WriteLine(defineResult);

---
#### Brainstorm ideas to address the problems using the "Ideate" step

In [ ]:
// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

var ideateResult = await kernel.RunAsync(defineResult.ToString(), skillDT["Ideate"]);

Console.WriteLine(ideateResult);
//display(ideateResult.ToString());

---
### Do all the steps in a Chaining Example

In [ ]:
#r "nuget: Markdig.Signed, 0.31.0"
using Microsoft.AspNetCore.Html;
using Markdig;
using System.Text.Json;
using System.Text.Json.Serialization;

// Load the Skills Directory
//var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
//var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");
/*
var input = @"
Customer 1: The power button on my phone is broken. The warranty is still valid.
Customer 2: My display stopped working.
Customer 3: The customer service rep didn't answer my email.
Customer 4: Every time I call customer support I get no answer.
Customer 5: The display screen cracked and it's still under warranty.
Customer 6: My power button fell off the phone. That's ridiculous.
Customer 7: I'm so frustrated with this company.
Customer 8: When I use the power button too much, it stops working.
";
*/

var myResult = await kernel.RunAsync(input, skillDT["Empathize"], skillDT["Define"], skillDT["Ideate"]);

var myPipeline = new MarkdownPipelineBuilder().UseAdvancedExtensions().Build();
var myResult2HTML = Markdown.ToHtml("### Ideate: 'Design Thinking' directions generated from customer empathy (Empathize) and problem definition (Define)\n\n" + myResult.ToString(), myPipeline);
var myHTMLContent = new HtmlContentBuilder();

myHTMLContent.AppendHtml(myResult2HTML);
myHTMLContent